<a href="https://colab.research.google.com/github/brianramos/bots/blob/master/EntropicTSP_SensitivityTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.graph_objects as go
import random
import math
import time
import statistics
import numpy as np

class City:
    def __init__(self, x, y, name=None):
        self.x = x
        self.y = y
        self.name = name if name else str(id(self))
        self.visited = False
        self.entropy = 1.0  # Initial entropy

def calculate_global_entropy(cities, unvisited_cities):
    """Calculates global entropy based on the distribution of unvisited cities."""
    # Example using a grid-based approach:
    width = max(city.x for city in cities)
    height = max(city.y for city in cities)

    # Dynamically adjust num_cells based on width or height (example)
    num_cells = int(max(width, height) / 10) + 1  # Ensure grid covers the city space

    # Create a grid with dimensions of 'num_cells x num_cells'
    grid = [[0] * num_cells for _ in range(num_cells)]

    cell_width = width / num_cells
    cell_height = height / num_cells

    for city in unvisited_cities:
        # Ensure cell_x and cell_y are within the grid boundaries
        cell_x = int(city.x / cell_width)
        cell_y = int(city.y / cell_height)

        # Prevent cell_x and cell_y from exceeding the grid index range
        cell_x = min(cell_x, num_cells - 1)
        cell_y = min(cell_y, num_cells - 1)

        grid[cell_x][cell_y] += 1

    # Calculate entropy based on cell counts
    total_cities = len(unvisited_cities)
    entropy = 0
    for row in grid:
        for count in row:
            if count > 0:
                probability = count / total_cities
                entropy -= probability * math.log(probability, 2)

    return entropy

def euclidean_distance(city1, city2):
    return math.sqrt((city1.x - city2.x)**2 + (city1.y - city2.y)**2)

def generate_tsp_problem(num_cities, width, height):
    cities = []
    for i in range(num_cities):
        cities.append(City(random.randint(0, width), random.randint(0, height), name=f"City {i}"))
    return cities

def path_length(path):
    length = 0
    for i in range(len(path) - 1):
        length += euclidean_distance(path[i], path[i+1])
    length += euclidean_distance(path[-1], path[0])  # Close the loop
    return length

def nearest_neighbor(cities):
    num_cities = len(cities)
    path = []
    current_city = random.choice(cities)
    current_city.visited = True
    path.append(current_city)

    for _ in range(num_cities - 1):
        min_distance = float('inf')
        next_city = None
        for city in cities:
            if not city.visited:
                distance = euclidean_distance(current_city, city)
                if distance < min_distance:
                    min_distance = distance
                    next_city = city

        if next_city:
            next_city.visited = True
            path.append(next_city)
            current_city = next_city
    return path

def two_opt(cities):
  num_cities = len(cities)
  path = cities.copy()
  improved = True
  while improved:
    improved = False
    for i in range(1, num_cities - 2):
        for j in range(i + 1, num_cities):
            if j - i == 1: continue  # changes nothing, skip
            new_path = path[:i] + path[i:j][::-1] + path[j:]
            if path_length(new_path) < path_length(path):
                path = new_path
                improved = True
                break  # improvement found, restart inner loop
        if improved:
            break  # improvement found, restart outer loop
  return path


def calculate_density(city, cities, radius):
    """Calculates the density of cities within a given radius around a city."""
    count = 0
    for other_city in cities:
        if other_city != city and euclidean_distance(city, other_city) <= radius:
            count += 1
    return count

def adaptive_contract_bubble(city, stretch, bubble_strength=0.5, density_radius=20, cities=None):  # Added cities argument
    """Contracts the bubble adaptively based on city density."""

    density = calculate_density(city, cities, density_radius)

    # Adjust bubble_strength based on density (example)
    adjusted_bubble_strength = bubble_strength * (1 - density / len(cities))

    city.entropy = max(0, city.entropy - (0.1 + 0.05 * stretch) * adjusted_bubble_strength)

def entropic_distance(city1, city2, stretch, initial_distances, global_entropy, bubble_sigma): # Added bubble_sigma
    """Calculates entropic distance, incorporating global entropy and bubble sigma."""
    initial_distance = initial_distances[(city1, city2)]
    entropy_factor = 1 / (city1.entropy * city2.entropy + 1e-10)
    stretch_factor = 1 + 0.1 * stretch
    global_entropy_weight = 1 + (global_entropy * 0.2)

    # Incorporate bubble_sigma (example - you can adjust how it influences distance)
    sigma_influence = math.exp(-(city1.entropy + city2.entropy) / (2 * bubble_sigma**2))
    return initial_distance * entropy_factor * stretch_factor / global_entropy_weight * sigma_influence


def traveling_salesperson_entropic_adaptive(cities, bubble_strength=0.5, density_radius=20, bubble_sigma=1.0):  # Added bubble_sigma
    num_cities = len(cities)
    path = []
    initial_distances = {}
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distance = euclidean_distance(cities[i], cities[j])
            initial_distances[(cities[i], cities[j])] = distance
            initial_distances[(cities[j], cities[i])] = distance

    current_city = random.choice(cities)
    current_city.visited = True
    path.append(current_city)
    total_stretch = 0
    unvisited_cities = cities[:]  # Copy of cities
    for _ in range(num_cities - 1):
        global_entropy = calculate_global_entropy(cities, unvisited_cities)
        min_distance = float('inf')
        next_city = None
        for city in cities:
            if not city.visited:
                distance = entropic_distance(current_city, city, total_stretch, initial_distances, global_entropy, bubble_sigma) # Pass bubble_sigma
                if distance < min_distance:
                    min_distance = distance
                    next_city = city
        if next_city:
            total_stretch += euclidean_distance(current_city, next_city)
            adaptive_contract_bubble(current_city, total_stretch, bubble_strength, density_radius, cities)
            adaptive_contract_bubble(next_city, total_stretch, bubble_strength, density_radius, cities)
            next_city.visited = True
            path.append(next_city)
            current_city = next_city
            unvisited_cities.remove(next_city)
    return path, path_length(path)

def reset_cities(cities):
    for city in cities:
        city.visited = False
        city.entropy = 1.0

def tune_entropic_path(cities, iterations, bubble_strengths):
    """Tunes the Entropic Path method parameters.  Uses the Bard-Entropy heuristic."""
    # Core concept: Using entropy as a heuristic to guide city selection (Concept by Bard).
    best_path = None
    min_length = float('inf')
    best_params = None

    for _ in range(iterations):
        reset_cities(cities) # Reset here
        bubble_strength = random.choice(bubble_strengths)
        path, length = traveling_salesperson_entropic_adaptive(cities.copy(), bubble_strength=bubble_strength)
        if length < min_length:
            min_length = length
            best_path = path
            best_params = (bubble_strength)

    return best_path, min_length, best_params

def generate_parameter_range(min_val, max_val, num_steps):
    """Generates a list of parameter values within a specified range."""
    step_size = (max_val - min_val) / (num_steps - 1)
    return [min_val + i * step_size for i in range(num_steps)]

def sensitivity_analysis(cities, bubble_strength_range, density_radius_range, bubble_sigma_range, iterations=10):
    """Performs sensitivity analysis with dynamically generated parameter ranges."""

    bubble_strengths = generate_parameter_range(*bubble_strength_range)
    density_radii = generate_parameter_range(*density_radius_range)
    bubble_sigmas = generate_parameter_range(*bubble_sigma_range)

    results = []
    for bubble_strength in bubble_strengths:
        for density_radius in density_radii:
            for bubble_sigma in bubble_sigmas:
                lengths = []
                for _ in range(iterations):
                    reset_cities(cities)
                    _, length = traveling_salesperson_entropic_adaptive(
                        cities.copy(), bubble_strength, density_radius, bubble_sigma
                    )
                    lengths.append(length)
                avg_length = np.mean(lengths)
                std_dev = np.std(lengths)
                results.append({
                    "bubble_strength": bubble_strength,
                    "density_radius": density_radius,
                    "bubble_sigma": bubble_sigma,
                    "avg_length": avg_length,
                    "std_dev": std_dev,
                })
    return results

def visualize_sensitivity_analysis(results):
    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=[r["bubble_strength"] for r in results],
        y=[r["density_radius"] for r in results],
        z=[r["bubble_sigma"] for r in results],
        mode='markers',
        marker=dict(
            size=10,
            color=[r["avg_length"] for r in results],                # set color to avg_length
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8,
            colorbar=dict(title='Average Path Length')
        ),
        hovertemplate='Bubble Strength: %{x}<br>Density Radius: %{y}<br>Bubble Sigma: %{z}<br>Avg Length: %{marker.color}<br>Std Dev: %{customdata:.2f}<extra></extra>',
        customdata=[r["std_dev"] for r in results]  # Add standard deviation as custom data
    )])

    fig.update_layout(scene = dict(
                    xaxis_title='Bubble Strength',
                    yaxis_title='Density Radius',
                    zaxis_title='Bubble Sigma'),
                    margin=dict(l=0, r=0, b=0, t=0), title="Sensitivity Analysis")
    fig.show()

def run_gather_statistics_iteratively(initial_city_count, iteration_count, step_size, num_iterations_per_run):
    city_counts = []
    entropic_path_lengths = []
    nn_path_lengths = []
    two_opt_path_lengths = []
    entropic_runtimes = []
    nn_runtimes = []
    two_opt_runtimes = []

    for city_count in range(initial_city_count, initial_city_count + iteration_count * step_size, step_size):
        print(f"Running for {city_count} cities...")
        city_counts.append(city_count)

        # Gather Statistics for current city_count
        entropic_lengths_current = []
        nn_lengths_current = []
        two_opt_lengths_current = []
        entropic_runtimes_current = []
        nn_runtimes_current = []
        two_opt_runtimes_current = []
        for _ in range(num_iterations_per_run):
          cities = generate_tsp_problem(city_count, 100, 100)
          # Entropic Path
          start_time = time.time()
          best_path, min_length, _ = tune_entropic_path(cities, 9, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
          end_time = time.time()
          entropic_lengths_current.append(min_length)
          entropic_runtimes_current.append(end_time - start_time)

          # Nearest Neighbor
          start_time = time.time()
          reset_cities(cities)
          nn_path = nearest_neighbor(cities.copy())
          end_time = time.time()
          nn_lengths_current.append(path_length(nn_path))
          nn_runtimes_current.append(end_time - start_time)

          # 2-opt
          start_time = time.time()
          reset_cities(cities)
          two_opt_path = two_opt(cities.copy())
          end_time = time.time()
          two_opt_lengths_current.append(path_length(two_opt_path))
          two_opt_runtimes_current.append(end_time - start_time)

        entropic_path_lengths.append(statistics.mean(entropic_lengths_current))
        nn_path_lengths.append(statistics.mean(nn_lengths_current))
        two_opt_path_lengths.append(statistics.mean(two_opt_lengths_current))
        entropic_runtimes.append(statistics.mean(entropic_runtimes_current))
        nn_runtimes.append(statistics.mean(nn_runtimes_current))
        two_opt_runtimes.append(statistics.mean(two_opt_runtimes_current))


    # Create Plotly chart
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=city_counts, y=entropic_path_lengths, mode='lines+markers', name='Entropic Path Length'))
    fig.add_trace(go.Scatter(x=city_counts, y=nn_path_lengths, mode='lines+markers', name='Nearest Neighbor Path Length'))
    fig.add_trace(go.Scatter(x=city_counts, y=two_opt_path_lengths, mode='lines+markers', name='2-opt Path Length'))
    fig.update_layout(title='Scaling of TSP Algorithms', xaxis_title='Number of Cities', yaxis_title='Path Length')

    fig.show()

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=city_counts, y=entropic_runtimes, mode='lines+markers', name='Entropic Runtime'))
    fig2.add_trace(go.Scatter(x=city_counts, y=nn_runtimes, mode='lines+markers', name='Nearest Neighbor Runtime'))
    fig2.add_trace(go.Scatter(x=city_counts, y=two_opt_runtimes, mode='lines+markers', name='2-opt Runtime'))
    fig2.update_layout(title='Runtime of TSP Algorithms', xaxis_title='Number of Cities', yaxis_title='Runtime(seconds)')
    fig2.show()


# Example usage:
num_cities = 20
cities = generate_tsp_problem(num_cities, 100, 100)

# Define parameter ranges using min, max, and num_steps
bubble_strength_range = (0.2, 0.8, 10)  # min, max, num_steps
density_radius_range = (1, 30, 10)
bubble_sigma_range = (0.5, 3.5, 10)

results = sensitivity_analysis(cities, bubble_strength_range, density_radius_range, bubble_sigma_range, iterations=5)
visualize_sensitivity_analysis(results)
